# A demo for RASA NLU

The goal of this notebook is to show how RASA NLU can be used to classify intents and detect entities in text sequences based on very few training examples.

https://rasa.com/docs/rasa/nlu/about/


# Imports

In [3]:
! python -m pip install -U rasa_core==0.9.6 rasa_nlu[spacy]
! python -m spacy download en_core_web_md 
! python -m spacy link en_core_web_md en --force

import spacy
from spacy import displacy
from collections import Counter
import en_core_web_md
nlp = spacy.load('en')
import random
import sys
python = sys.executable

import rasa_nlu
import rasa_core
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

import logging, io, json, warnings

  Using cached https://files.pythonhosted.org/packages/72/de/82e762685a991351a9f89d934b1b25fe04d10462cf4017f93227cb4d9058/rasa_core-0.9.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/19/c4/c6146c445a17b6ce414d773f93c941c44ca16720609000ae3d01409f9dfb/rasa_nlu-0.15.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e7/9f/83bb34eaf84032b0b54fcc4a6aff1858572d279d65a301c7ae875f523df5/ruamel.yaml-0.15.100-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/08/0f/7877fc42fff0b9d70b6442df62d53b3868d3a6ad1b876bdb54335b30ff23/coloredlogs-10.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/3b/f6/7a76333cf0b9251ecf49efff635015171843d9b977e4ffcf59f9c4428052/redis-2.10.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f3/34/9ef20ed473c4fd2c3df54ef77a27ae3fc7500b16b192add4720cab8b2c09/APScheduler-3.6.3-py2.py3-none-any.whl
     |███████████████████████████

# Spacy Example

In [4]:
sentence = 'Add milk to my shopping list'
doc = nlp(sentence)
print([(X.text, X.label_) for X in doc.ents])

displacy.render(doc, style='dep', jupyter = True, options = {'distance': 120})
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

[]


Add add VERB VB ROOT Xxx True False
milk milk NOUN NN dobj xxxx True False
to to ADP IN prep xx True False
my -PRON- ADJ PRP$ poss xx True False
shopping shopping NOUN NN compound xxxx True False
list list NOUN NN pobj xxxx True False


# Examples to train RASA NLU

In [7]:
nlu_md = """
## intent:greet
- hey
- hello there
- good morning
- good evening
- hey there
- goodmorning
- goodevening
- good afternoon

## intent:bye
- good by
- good night
- good afternoon
- bye
- goodbye
- have a nice day
- see you around
- bye bye

##intent:add_item
- can you add [Milk](item) to the [shopping list](list)?
- can you add [Eggs](item) to the [shopping bag](list)?
- please add [bread](item) to my [shopping list](list)?
- please add [Mayo](item) to the [grocery list](list)?
- can you add [milk](item) to [shopping bag](list)?
- add [butter](item) to the [gorcery bag](list)?
- add [butter](item) to [gorcery bag](list)?
- add [milk](item)
- add [corn flakes](item)
- add [smarties](item)
- add [apples](item)
"""
%store nlu_md > nlu.md

Writing 'nlu_md' (str) to file 'nlu.md'.


# RASA Model

In [8]:
config = """
language: "en"

pipeline:
- name: "nlp_spacy" # loads the spacy language model
- name: "tokenizer_spacy" # splits the sentence into tokens
- name: "ner_crf" # uses the pretrained spacy NER model
- name: "intent_featurizer_spacy" # transform the sentence into a vector representation
- name: "intent_classifier_sklearn" # uses the vector representation to classify using SVM
- name: "ner_synonyms" # trains the synonyms
""" 
%store config > config.yml

Writing 'config' (str) to file 'config.yml'.


In [9]:
# loading the nlu training samples
training_data = load_data("nlu.md")

# trainer to educate our pipeline
trainer = Trainer(config.load("config.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:522: FutureWarning: Passing (type, 1) or 

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.0s finished


# Predict

In [15]:
# A helper function for prettier output

def pprint(o):   
    print(json.dumps(o, indent=2))
    
pprint(interpreter.parse("add spices to grocery list"))

{
  "intent": {
    "name": "add_item",
    "confidence": 0.8971480072107451
  },
  "entities": [
    {
      "start": 4,
      "end": 10,
      "value": "spices",
      "entity": "item",
      "confidence": 0.9591311383910901,
      "extractor": "CRFEntityExtractor"
    },
    {
      "start": 14,
      "end": 21,
      "value": "grocery",
      "entity": "list",
      "confidence": 0.4981706364567592,
      "extractor": "CRFEntityExtractor"
    }
  ],
  "intent_ranking": [
    {
      "name": "add_item",
      "confidence": 0.8971480072107451
    },
    {
      "name": "greet",
      "confidence": 0.05511977075763733
    },
    {
      "name": "bye",
      "confidence": 0.04773222203161763
    }
  ],
  "text": "add spices to grocery list"
}
